# Paper
## Imports
https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=10863523

bib reference -> 10863523

In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore

from src.config import *
from src.data import *
from src.models.efficientnet import EfficientNetB5Custom
from src.utils import *
from src.data import OriginalOAIDataset


# Class balance

In [ ]:
NEW_OAI_DATASET = 'dataset/mendeleyOAI_dataset/augmented_dataset_1'


In [ ]:
# Original dataset
data = explorar_split_data(MENDELEY_OAI_224_SPLIT_PATH)

In [ ]:
ORIGINAL_TRAIN_PATH = os.path.join(MENDELEY_OAI_224_SPLIT_PATH, 'train')
classes = [d for d in os.listdir(ORIGINAL_TRAIN_PATH) if os.path.isdir(os.path.join(ORIGINAL_TRAIN_PATH, d))]
data_gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
if not os.path.exists(NEW_OAI_DATASET):
    os.makedirs(NEW_OAI_DATASET)

augmentation_classes = [1, 2, 5, 10, 20]
for class_name in classes:
    class_dir = os.path.join(ORIGINAL_TRAIN_PATH, class_name)
    print(int(class_name))
    num_augmentations = augmentation_classes[int(class_name)]
    print(f"Generando imágenes aumentadas para la clase {class_name}...")
    print(f"Se generarán {num_augmentations} imágenes")
    print(f"Directorio original de la clase: {class_dir}")
    imagenes_generadas = 0
    for img_name in os.listdir(class_dir):
        
        
        img_path = os.path.join(class_dir, img_name)
        img = cv2.imread(img_path)
        
        # Verificar si la imagen fue leída correctamente
        if img is None:
            print(f"Error al leer la imagen {img_path}. Puede que no sea una imagen válida o esté dañada.")
            continue
        
        # Convertir la imagen a un numpy array
        img_array = np.array(img)
        img_array = img_array.reshape((1,) + img_array.shape)  # Añadir dimensión batch
        # Generar imágenes aumentadas
        """
        for i in range(num_augmentations):
            for batch in data_gen.flow(img_array, batch_size=1, save_to_dir=NEW_OAI_DATASET, save_prefix='aug', save_format='png'):
                break
        """
        imagenes_generadas += num_augmentations

    print(f"Se han generado {imagenes_generadas} imágenes aumentadas para la clase {class_name}\n-----------------------------------\n")

# Train Model

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
])
BATCH_SIZE = 32

train_dataloader = OriginalOAIDataset(mode='train', transform=transform,local=True).get_dataloader(batch_size=BATCH_SIZE)

model = EfficientNetB5Custom(num_classes = 5)

Local mode, 100 images loaded
Local mode, 100 images loaded
Local mode, 100 images loaded
Local mode, 100 images loaded
Local mode, 100 images loaded


c:\Users\34658\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\34658\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B5_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B5_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
batch = next(iter(train_dataloader))


4
4
3
3
1
0
2
4
0
2
4
0
0
0
0
3
3
3
1
1
3
2
2
0
1
4
0
2
4
1
3
1


In [10]:
batch[0].shape

torch.Size([32, 1, 224, 224])

In [ ]:
image = Image.open(data[0][0])
transform(image)